# 🔽 Extração e Preparação do Dataset `trn.json` do Google Drive
Este notebook realiza as seguintes etapas:
1. Instala o pacote `gdown` caso não esteja instalado;
2. Baixa um arquivo `.zip` do Google Drive;
3. Extrai apenas o arquivo `trn.json` de dentro do `.zip`;
4. Remove o `.zip` para economizar espaço em disco;
5. Converte o `trn.json` para o formato `.jsonl` com campos `prompt` e `completion`.


In [1]:
# Instala o gdown automaticamente se necessário
import subprocess, sys

try:
    import gdown
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gdown"])
    import gdown

## 📁 Baixando o ZIP do Google Drive
Vamos usar o `gdown` para baixar o arquivo `.zip` diretamente de um link público do Google Drive.


In [2]:
zip_file_id = "12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK"
zip_output = "dados.zip"

# Baixa o arquivo ZIP
gdown.download(f"https://drive.google.com/uc?id={zip_file_id}", zip_output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK
From (redirected): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK&confirm=t&uuid=f585b628-e3b6-4bc3-81fa-21c27c0e8dfc
To: /content/dados.zip
100%|██████████| 890M/890M [00:13<00:00, 66.8MB/s]


'dados.zip'

## 📦 Extraindo apenas o `trn.json` do ZIP
Usamos o módulo `zipfile` da biblioteca padrão para extrair somente o arquivo desejado.


In [4]:
import zipfile
import os
import shutil
import gzip

caminho_zip = "dados.zip"
texto_desejado = "trn.json"
extraido = None
nome_final = None

# Etapa 1: Abrir e localizar arquivo com 'trn.json'
with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
    for arquivo in zip_ref.namelist():
        if texto_desejado in os.path.basename(arquivo):
            zip_ref.extract(arquivo)
            nome_extraido = os.path.basename(arquivo)
            shutil.move(arquivo, nome_extraido)
            extraido = nome_extraido
            print(f"✅ Arquivo extraído: {extraido}")
            break

# Etapa 2: Descompactar .gz se necessário
if extraido:
    if extraido.endswith('.gz'):
        nome_final = extraido.replace('.gz', '')
        with gzip.open(extraido, 'rb') as f_in, open(nome_final, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
        os.remove(extraido)
        print(f"📂 Arquivo GZ descompactado para: {nome_final}")
    elif extraido.endswith('.json'):
        nome_final = extraido
        print(f"📄 Arquivo JSON pronto para uso: {extraido}")
    else:
        print(f"⚠️ Extensão desconhecida: {extraito}")
else:
    print("❌ Nenhum arquivo contendo 'trn.json' foi encontrado.")

# Etapa 3: Limpeza de arquivos e pastas desnecessárias (exceto sample_data)
if nome_final:
    for item in os.listdir():
        if item != nome_final and not item.endswith('.ipynb') and item != 'sample_data':
            item_path = os.path.join(os.getcwd(), item)
            try:
                if os.path.isfile(item_path):
                    os.remove(item_path)
                    print(f"🗑️ Arquivo removido: {item}")
                elif os.path.isdir(item_path):
                    shutil.rmtree(item_path)
                    print(f"🗑️ Pasta removida: {item}")
            except Exception as e:
                print(f"⚠️ Erro ao remover {item}: {e}")


✅ Arquivo extraído: trn.json.gz
📂 Arquivo GZ descompactado para: trn.json
🗑️ Pasta removida: .config
🗑️ Pasta removida: LF-Amazon-1.3M
🗑️ Arquivo removido: dados.zip


## 📄 Função para preparar o JSONL
Transformamos o `trn.json` em um arquivo `.jsonl` contendo `prompt` e `completion`, ideal para fine-tuning de modelos de linguagem.


In [5]:
import json

def preparar_dataset_jsonl(caminho_entrada, caminho_saida):
    contador = 0
    ignorados = 0

    with open(caminho_entrada, 'r', encoding='utf-8') as f_in, \
         open(caminho_saida, 'w', encoding='utf-8') as f_out:

        for linha in f_in:
            try:
                item = json.loads(linha)
                titulo = item.get("title", "").strip()
                descricao = item.get("content", "").strip()

                if titulo and descricao:
                    prompt = f"O que é {titulo}?"
                    completion = descricao
                    f_out.write(json.dumps({"prompt": prompt, "completion": completion}, ensure_ascii=False) + '\n')
                    contador += 1
                else:
                    ignorados += 1

            except json.JSONDecodeError:
                ignorados += 1
                continue

    print(f"✅ Dataset gerado com sucesso!")
    print(f"📄 Linhas válidas: {contador}")
    print(f"🚫 Linhas ignoradas (sem conteúdo ou erro): {ignorados}")
    print(f"💾 Arquivo salvo em: {caminho_saida}")

## ▶️ Executando a Preparação dos Dados


In [ ]:
CAMINHO_JSON = "trn.json"
SAIDA_JSONL = "dataset_preparado.jsonl"
preparar_dataset_jsonl(CAMINHO_JSON, SAIDA_JSONL)